In [1]:
# !conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7x6qs9j_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7x6qs9j_
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
Using cached nvidia_cusp

In [2]:
import torch
import clip
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(clip.available_models())


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [13]:
model, preprocess = clip.load("RN50x64", device=device)


In [14]:
task_description = ["A bookshelf placed against a wall"]

upper_thresh = 28
lower_thresh = 2

# Load and preprocess user-uploaded image
image = preprocess(Image.open("/content/upload2.jpg")).unsqueeze(0).to(device)


In [15]:
# Convert text to CLIP embeddings
text_tokens = clip.tokenize(task_description).to(device)

# Extract features
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)

# Normalize features
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

# Compute cosine similarity between text and image
similarity_score = (text_features @ image_features.T).item()
print(f"Similarity Score: {similarity_score}, Match: {similarity_score*100 > sum([upper_thresh,lower_thresh])/2}")


Similarity Score: 0.289306640625, Match: True


In [16]:
model_path = '/content/clip_model.pt'

torch.save(model,model_path)

# Save the model as TorchScript
# scripted_model = torch.jit.script(model)
# torch.jit.save(scripted_model, model_path)
